In [1]:
import cobra
import GEMS
import pandas as pd
import numpy as np
import os
import re

import pickle

from etcpy import etc
from etcpy import tempDep

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

In [2]:
data = GEMS.load_exp_data('../data/ExpGrowth.csv')
path = '../'
params = pd.read_csv(os.path.join(path,'data/model_enzyme_params_new_tagged.csv'),index_col=0)
df = etc.calculate_thermal_params(params)

dfae_batch = data[0].set_index('Ts').rename_axis(None)
params

,Topt,Topt_std,Length,Tm,Tm_std,T90,dCpt,dCpt_std,topt_source,TmTag
P77467,306.15,10,262,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76077,311.15,10,309,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76078,310.15,10,95,328.717325,7.588735,NaN,-4000,1000,BullShit,Mean
P76079,309.15,10,248,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76081,310.15,10,356,328.717325,7.588735,NaN,-4000,1000,BullShit,Mean
...,...,...,...,...,...,...,...,...,...,...
P0AG80,313.15,10,438,329.166303,4.070000,330.45,-4000,1000,predicted,Exp
P77718,307.15,10,482,319.125751,4.070000,320.15,-4000,1000,predicted,Exp
P0A6D5,308.15,10,288,328.762604,7.710000,NaN,-4000,1000,predicted,Mean
standard,308.15,10,272,328.762604,7.710000,NaN,-4000,1000,BullShit,Mean


In [3]:
def aerobic(params):
    # thermalParams: a dictionary with ids like uniprotid_Topt 
    df = etc.calculate_thermal_params(params)
    #mae = pickle.load(open(os.path.join(path,'models/aerobic.pkl'),'rb'))
    mae = cobra.io.load_matlab_model('../models/ecModel.mat')
    
    match = re.compile('usage_prot_*', re.IGNORECASE)
    # for react in mae.reactions:
    #     if match.match(react.id) and react.lower_bound != -1000:
    #         ori_lb = react.lower_bound
    #         react.lower_bound = ori_lb*2.5


    mae.reactions.EX_glc__D_e.lower_bound = -100
    mae.reactions.BIOMASS_Ec_iJO1366_core_53p95M.upper_bound = 50
    #mae.medium['EX_glc__D_e'] = 100
    
    try: 
        rae, gae, oae, cae, aae = etc.simulate_growth(mae,dfae_batch.index+273.15,df=df,sigma=0.5)
    except: 
        rae = np.zeros(dfae_batch.shape[0])
        model = None
    
    rae = [0 if x is None else x for x in rae]
    rae = [0 if x<1e-3 else x for x in rae]
    rexp = dfae_batch['r'].values
    #print(rae)
    print(abs(rae - dfae_batch['r'].values))
    print('r2_batch:',r2_score(rexp,rae))
    print('MSE_ae',MSE(rexp,rae))
    return {'growth':np.array(rae), 'glucose':np.array(gae), 'oxygen': np.array(oae), 'carbonDioxide': np.array(cae), 'acetate': np.array(aae)}, r2_score(rexp,rae), MSE(rexp,rae)

In [4]:
df = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])

for i in range(1,11):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_increasing_topt(params, ['Tm'])
    rae, score, mse = aerobic(newParams)
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-06
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.12459015549619959
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.19840275675885896
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.32843410516713295
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.4401049645567501
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.5696016151683656
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.6086580601947323
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.5724977805343204
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  0.5054040935847728
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  0.4548117937213355
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  0.41129654931090615
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  0.3580438065885198
NGAM is: 6.9000236503978725
Growth at  44.45349

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  45.28989924561603 is:  0
NGAM is: 7.088333029723301


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  46.64533098816804 is:  0
NGAM is: 7.226932905456837


...

Failed to solve the problem
Growth at  45.28989924561603 is:  0
NGAM is: 7.088333029723301


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  46.64533098816804 is:  0
NGAM is: 7.226932905456837


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  48.47717620988101 is:  0
[0.0128701  0.21113424 0.16429146 0.25548854 0.38315644 0.48292729
 0.6130955  0.81075108 0.98422906 0.90730223 1.03163423 1.08433436
 1.007028   0.922572   0.16272   ]
r2_batch: -2.2375337918380263
MSE_ae 0.49826113741463124
10
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.24268976354021965
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.32967739560301385
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.4428576128001552
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.5095736897776401
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.5682817196242301
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.6091800419436701
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.5725104971177034
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  0.505388349915528
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  0.454777796200384
NGAM is: 6.

In [5]:
df.to_csv('../results/wigTm.csv')

In [6]:
df

,iter,temp,growth,o2flux,gluFlux,co2flux,acetateFlux,R2,MSE
0,1,20.153299,0.124590,3.049121,3.714325,-0.337649,-5.439709,-1.960395,0.455609
1,1,23.001511,0.198403,3.613692,4.839528,-0.337850,-6.578387,-1.960395,0.455609
2,1,26.331896,0.328434,6.484301,6.433398,-4.146152,-7.447276,-1.960395,0.455609
3,1,28.251820,0.440105,9.355476,7.369233,-9.285463,-7.423011,-1.960395,0.455609
4,1,29.994212,0.569602,14.205571,7.964842,-15.420984,-4.492405,-1.960395,0.455609
...,...,...,...,...,...,...,...,...,...
10,10,42.238764,0.357982,10.509632,4.745845,-11.273492,-1.252867,-1.872203,0.442036
11,10,44.453491,0.296655,9.111899,4.045862,-9.744899,-1.169823,-1.872203,0.442036
12,10,45.289899,0.274149,8.595059,3.790610,-9.180038,-1.148912,-1.872203,0.442036
13,10,46.645331,0.239774,7.797615,3.402498,-8.309243,-1.126286,-1.872203,0.442036
